## Prevendo os preços do Petróleo Brent do IPEA

In [6]:
import pandas as pd
from plotly import graph_objs as go
from utils import atualizando_dados_ipea

In [2]:
path = atualizando_dados_ipea()
dados = pd.read_csv(path)

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11092 entries, 0 to 11091
Data columns (total 2 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Data                                  11092 non-null  object 
 1   Preço - petróleo bruto - Brent (FOB)  11092 non-null  float64
dtypes: float64(1), object(1)
memory usage: 173.4+ KB


In [4]:
dados.tail()

,Data,Preço - petróleo bruto - Brent (FOB)
11087,2024-01-16,80.15
11088,2024-01-17,78.88
11089,2024-01-18,81.04
11090,2024-01-19,80.71
11091,2024-01-22,81.70


In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dados['Data'], y=dados['Preço - petróleo bruto - Brent (FOB)'], name="Preço do Petróleo Brent"))
fig.layout.update(title_text='Preço do Petróleo Brent (FOB)', xaxis_rangeslider_visible=True)
fig.show()   


### Prophet

In [8]:
from prophet import Prophet
from prophet.plot import plot_plotly
import pickle

In [9]:
periodo = 7

df_train = dados[['Data','Preço - petróleo bruto - Brent (FOB)']]
df_train = df_train.rename(columns={"Data": "ds", "Preço - petróleo bruto - Brent (FOB)": "y"})

m = Prophet()
m.fit(df_train)
future = m.make_future_dataframe(periods=periodo, freq="B")
forecast = m.predict(future)

15:11:35 - cmdstanpy - INFO - Chain [1] start processing
15:11:43 - cmdstanpy - INFO - Chain [1] done processing


In [10]:
forecast.tail(periodo)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
11092,2024-01-23,89.410246,74.129469,99.811094,89.410246,89.410246,-2.490511,-2.490511,-2.490511,-0.087972,-0.087972,-0.087972,-2.402539,-2.402539,-2.402539,0.0,0.0,0.0,86.919735
11093,2024-01-24,89.425859,74.162595,99.624904,89.425859,89.425859,-2.536242,-2.536242,-2.536242,-0.133727,-0.133727,-0.133727,-2.402515,-2.402515,-2.402515,0.0,0.0,0.0,86.889617
11094,2024-01-25,89.441471,74.300439,99.025612,89.441471,89.441471,-2.452046,-2.452046,-2.452046,-0.047567,-0.047567,-0.047567,-2.404479,-2.404479,-2.404479,0.0,0.0,0.0,86.989425
11095,2024-01-26,89.457084,74.351884,98.824148,89.457084,89.457084,-2.435612,-2.435612,-2.435612,-0.027852,-0.027852,-0.027852,-2.407760,-2.407760,-2.407760,0.0,0.0,0.0,87.021471
11096,2024-01-29,89.503921,73.826685,99.407627,89.503921,89.503921,-2.431438,-2.431438,-2.431438,-0.013958,-0.013958,-0.013958,-2.417481,-2.417481,-2.417481,0.0,0.0,0.0,87.072483
11097,2024-01-30,89.519534,74.056288,98.508496,89.519534,89.519534,-2.505655,-2.505655,-2.505655,-0.087972,-0.087972,-0.087972,-2.417683,-2.417683,-2.417683,0.0,0.0,0.0,87.013879
11098,2024-01-31,89.535146,74.559851,99.132885,89.535146,89.535146,-2.548497,-2.548497,-2.548497,-0.133727,-0.133727,-0.133727,-2.414770,-2.414770,-2.414770,0.0,0.0,0.0,86.986649


In [17]:
with open('modelo/prophet.pkl', "wb") as f:
    pickle.dump(m, f)

In [ ]:
m1 = pickle.load(open('modelo/modelo_prophet.pkl', 'rb'))
plot_plotly(m1, forecast)

In [14]:
import joblib

In [16]:
joblib.dump(m, 'modelo/prophet.joblib')

['modelo/prophet.joblib']

### Modelo 1